In [14]:
# CONFIG File 
# Data annotations and data set configurations
assay_list_location = "/content/drive/My Drive/Imperial college London/alla.csv"
dataset_path_html = r"/content/drive/My Drive/Imperial college London/data" # Processed raw html with  beautifulsoup 
dataset_path_json = "" # Processed with autoscoopyst
# PROCCESED data
train_data_path="" # Where to save the JSON file with all the annontated data


# Spacy model 
TRAIN_DATA ="" # JSON file with final output 
test_data=""


In [15]:
#1
#Importing drive contents 
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
#Read assay list
assay=[]
df = pd.read_csv("/content/drive/My Drive/Imperial college London/alla.csv",dtype= str) # READ CSV AS STRING !!!
assay = df.values.tolist()
assay = list(map(''.join, assay)) # convert list of lists to list of strings 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ParserError: ignored

In [13]:
#2
def html_loader(path=dataset_path_html):  
  ''' 
  Reads html files and makes the text more readable 
  using beautiful soup's get text method.
  Returns a list of strings, where each element is one html file's text
  '''
  import os
  from bs4 import BeautifulSoup
  #import urllib2
  content=[]
  #url = "/content/drive/My Drive/Imperial college London/data/PMC3908470.html"
  for filename in os.listdir(path):
      if filename.endswith(".html"):
          fullpath = os.path.join(path, filename)
          page = open(fullpath, errors='ignore') # encoding='mac_roman' because the files were created in mac , errors ignore if it can't open some files
          soup = BeautifulSoup(page)
          [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
          soup=soup.get_text()
          items = ' '.join(soup.split())
          content.append(items)
  print("Number of papers processed: ",len(content))
  return content
#print(html_loader())  
content=html_loader()

  

NameError: ignored

In [12]:
#3
# Easy way but computentianlly expensive
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.max_length=7000000
#txt = '''I really like NMR spectroscopy. Electrospray ionization is my favorite form of ionizing things because it can be very versatile, and as such Electrospray ionization is widely used. The word chroma comes from the greek word for color, that is why Gas chromatography is called like that!'''
#for i in range(0, len(content)):
text = ''.join([str(elem) for elem in content]) 
#txt= train
#print(txt)
#txt= "This is my NMR i like it. This is a random sentence. This is NMRSQPREWE, to try and confuse the model. I have some QQ wanna try ?"
tokens = nlp(text, disable=['ner'])
sentences=[]
for sent in tokens.sents:
    sentences.append(sent.string.strip())
print(sentences)


NameError: ignored

In [20]:
#4
# Beta V 1.1
# IN PROGRESS
# Version incldes finding nouns and then doing the matching 
# Efficient vs non Efficient way
'''
This code reads into a list all the assay methods from a csv file.
A function takes a text as an input an a python list of assays. 
The function performs matching of all sentences that contain a word which 
is in the list. This is done using regular expression, the word is found 
and then the expression looks for puntcuation signs left from the word and 
right from the word so that it can extract the sentence which contains the word.

The user can choose the given assay when running the function, 
or if he doesn't input a parameter it will go through the list of all assays in the file.

'''
#from google.colab import drive
#drive.mount('/content/drive')

import re #time complexity issues of finite state machines ?
import pandas as pd
import spacy
import numpy as np

assay=[]
df = pd.read_csv("/content/drive/My Drive/Imperial college London/alla.csv",dtype= str) # READ CSV AS STRING !!!
assay = df.values.tolist()
assay = list(map(''.join, assay)) # convert list of lists to list of strings 


def finder(text,user_assay):    
    pattern1=r'[^.?!]*(?<=[.?\s!])%s(?=[\s.?!])[^.?!]*[.?!]' #extracts full sentences that contain a word 
    pattern2=r'\b%s\b' #extracts a given word 
    index=[]
    for i in range(len(assay)):
      tmp=re.findall(pattern2 %assay[i],text)  
      if (len(tmp)>0):
       index.append(i)
    res_list = [assay[j] for j in index]
    print("The assays mentioned are: \n ", res_list, type(res_list))
    return res_list
    '''
     #if the user chooses a custom assay not from the list
     # One big if is needed to check whether user_assay is None or not
    else:  
     out2=re.findall(pattern % user_assay , text, re.MULTILINE )
     if (len(out2) == 0):
      print("No matching assays")
     else:
      output.append(out2) 
      print("At least one match was found :")
      print('\n'.join(out2))
   '''

# Run the program outside of the function  
# Expected assay output: NMR spectroscopy, liquid chromatography, (LC)-mass spectrometry, gas chromatography-mass spectrometry, 1H NMR spectroscopy, NMR spcetrospcy  





x=finder(text,"")

#x=finder(text,"") # You should write "" instead of None for the input of the function
'''
nlp=spacy.load("en_core_web_sm")
doc=nlp(text)
nouns=list(doc.noun_chunks)
nouns=''.join(str(nouns)) # putting the nouns in a string because my function takes in a string
nouns=nouns.replace(',','.') # I join them with dots because I tried to fix the regex to recognize commas but it didn't work (argument agains regex)
in_txt=sentences # copy the list sentences to another list in_txt - sentences keeps intackt as a list of the sentences
in_txt=''.join(str(in_txt)) # convert in_txt to string by joining it from self, I need it all as one string for the finder function 
# I keep sentences as a list but create a string which is my input text becuase I need a string for it to be readable with the regex, but a list of the loops
#x=finder(in_txt,"") # You should write "" instead of None for the input of the function 
# x is a list of the assays found in the text , 
# in_txt is only needed to find the mentioned assays 
# Find span of all texts
#Let's see if it works with overlapping matches 
'''


The assays mentioned are: 
  ['(1)H NMR', 'NMR', '1H-NMR', '1H-NMR', 'NMR spectroscopy', '13C NMR', '13C NMR', 'COSY', 'CPMG', 'Carr-Purcell-Meiboom-Gill', 'Mass spectrometry', 'mass spectrometer', 'MS', 'tandem mass spectrometry', 'MS/MS', 'FID', 'Liquid chromatography', 'LC-MS', 'LC/MS', 'UPLC-MS', 'HPLC-MS', 'Ultra Performance LC', 'reverse-phase HPLC', 'reversed-phase HPLC', 'RP', 'LC-MS/MS', 'Electrospray ionization', 'ESI', 'TOF', 'QTOF', 'QqQ', 'DI', 'Orbitrap', 'quadrupole ion trap', 'SPE', 'Lipidomics', 'Thermo', 'Waters', 'Agilent', 'Thermo Fisher', 'Bruker', 'Varian', 'SCIEX'] <class 'list'>


'\nnlp=spacy.load("en_core_web_sm")\ndoc=nlp(text)\nnouns=list(doc.noun_chunks)\nnouns=\'\'.join(str(nouns)) # putting the nouns in a string because my function takes in a string\nnouns=nouns.replace(\',\',\'.\') # I join them with dots because I tried to fix the regex to recognize commas but it didn\'t work (argument agains regex)\nin_txt=sentences # copy the list sentences to another list in_txt - sentences keeps intackt as a list of the sentences\nin_txt=\'\'.join(str(in_txt)) # convert in_txt to string by joining it from self, I need it all as one string for the finder function \n# I keep sentences as a list but create a string which is my input text becuase I need a string for it to be readable with the regex, but a list of the loops\n#x=finder(in_txt,"") # You should write "" instead of None for the input of the function \n# x is a list of the assays found in the text , \n# in_txt is only needed to find the mentioned assays \n# Find span of all texts\n#Let\'s see if it works with

In [62]:
print(type(content))

<class 'str'>


In [27]:
#5
# INPUT: List x of all mentioned assays , List sentences of all sentences in the text
#=============================================================================


#Saving to a JSON file that is compatible with Spacy NER
#import json 
label="ASSAY" # za ako nema entity isto bitno e !!, i ke treba starts i ends za sekoja recenica, ako se prazni is okay
# ako e prva staj zagrada ako ne, nishto
# Try to save in dictironary, each sentence has a seperate dictionary
# Za ova so dolnava crta da naucam bolje kako se praj sho kako

dicts = [{} for _ in range(len(sentences))] # create a list of dictionaries that has the same number of diconaries as sentences 
for k in range (len(sentences)):
  d={"content":None,"entities":[]} # Specify the dictionary format 
  dicts[k].update(d)              # Write an empty list of dictionaries 
# Functions to append multiple values to a key in dictionary

def add_content_in_dict(sample_dict, key, list_of_values):
    """Append multiple values to a key in the given dictionary"""
    if key not in sample_dict:
        sample_dict[key] = ""
    sample_dict[key]=(list_of_values) # because it initializes, it replaces the first key value pair
    return sample_dict

def add_entites_in_dict(sample_dict, key, list_of_values):
    """Append multiple values to a key in the given dictionary"""
    if key not in sample_dict:
        sample_dict[key] = []
    #if sample_dict.get(key)== []:
        #sample_dict[key]=[list_of_values]
    sample_dict[key].append(list_of_values)  # it adds to the first key value pair
    return sample_dict

# This loop finds the matches "found entities in ALL sentences" saved in the output of the finder function which is a list x[]
# It looks for the span of matches and the m
# OVERLAPPING ISSUE FIX
s=0
e=0
for k in range(len(sentences)): # cycle through sentences
  #print("Matches in" , k , "sentence:")
  add_content_in_dict(dicts[k],'content',sentences[k])
  for j in range(len(x)): # look for all entites in a given sentence # LEN X VO ZAGRADATA ; len(x) e ustvari dolzianta na toj celiot tekst od funkcijta sho iskacha od finder
    for m in re.finditer(x[j], sentences[k]):  # cycle through all charaters in a given sentence with a given entity
        #print(m.start(), m.end(), m.group()) # print the starting char, ending char , and match IN THE given sentence
        #temp=m.end() - m.start()
        if (m.end() - m.start() > e - s):
          add_entites_in_dict(dicts[k],'entities',(m.start(),m.end(),label)) # updatni entities za TOJ istiot content    
          e=m.end()
          s=m.start()
          continue
        else:
          add_entites_in_dict(dicts[k],'entities',(s,e,label))
          e=m.end()
          s=m.start()
          continue
        
        

        #[19,30,"subject"]
        #add_entites_in_dict(dicts[k],'entities',[])
        #starts.append(m.start())  # add to a list of all starting characters
        #ends.append(m.end())      # add to a list of all ending charachters 

#starts.sort()  # sort the lists because I need the pairs of elements to find the span of the match
#ends.sort()    # I will first try with overlapping entites but I might also correct that 
#print(starts,ends)

# Try to save in dictironary, each sentence has a seperate dictionary
# Za ova so dolnava crta da naucam bolje kako se praj sho kako

# Granicniot slucaj za koa nema najdeno entities sho da stai
fulls=[]   # list of all the data with found entities  
empties=[]  # list of all the data with empty entities 
fin=[]   # final list with 80% fulls and 20% of empties

dr = [{} for _ in range(len(sentences))] #
for k in range (len(sentences)):
  d={'entities':dicts[k].get('entities')}
  dr[k].update(d) 



for i in range (len(sentences)):
  elem=(dicts[i].get('content'),dr[i])
  if (dr[i].get('entities')!=[]):  # append only the ones that have enttites in them
   fulls.append(elem)
  else:
    empties.append(elem)


#====================================
#
# TUKA DA SE STAVI FUNKCIJATA NEGDE
#
#======================================


if (len(empties)<=(len(fulls)//10)):
  fin = fulls + empties
else:
  fin = fulls + empties[(len(fulls)//10):len(fulls)//5]   # I put 10% of the empties beginnign from the first 10 percent and until the second 10%
# fin  =   [  (get.key, {'entities': [(),()]}) , (get.key, {'entities': [(),()]})   ]
#dicts[0].get('content') # ova e getkey
#print(len(fin))
print(fin)


import json

with open('test.json', 'w') as fout:
    json.dump(fin , fout, sort_keys=True, indent=1 ) # save json in pretty format


with open('test.json', "r") as read_file:   # read json file 
    data_train = json.load(read_file) 
print(data_train)


NameError: ignored

In [62]:
# 21 load files and clear out
from google.colab import drive
drive.mount('/content/drive')
import json
with open('/content/drive/My Drive/Imperial college London/test_1.json', "r") as read_file:   # read json file 
    data_train = json.load(read_file) 

test=data_train
tmp=[]

for i in range (0, len(test)):
  tmp.append(test[i][1].get('entities')) # [[3, 6, 'ASSAY'], [0, 6, 'ASSAY'], [0, 6, 'ASSAY']]  ;   [[111, 113, 'ASSAY']]


# we creating a list called tmp, it will have all the entities, each element is one sentence. 
# tmp[i] is the input to the function 
'''
the function would return what tmp[i] should have --> data train[i][1].get('entities'))
'''

#print(tmp[0][0][0]) # this one is 3 ;  1. which list 2. whcih element in the list 3. 0 - start 1- ends - 2- label 




def jac(args):
  '''
  input is [[3, 6, 'ASSAY'], [0, 6, 'ASSAY'], [0, 6, 'ASSAY']] -> matches in first sentence
  It should be ran for each sentence!
  If the function had variable parameters 
  *args should be put instead of args and read as follows:
  lis=[]
  for arg in args:
    lis.append(arg)
  print(lis[0][0])
  ''' 
  lis=[]
  lis.append(args)
  # lis = [[3, 6, 'ASSAY'], [0, 6, 'ASSAY'],[0, 6, 'ASSAY']]
  '''
  # In order to use set theory operations we first 
  # make a list  of lists, where each element is a list 
  # with elements between two values e.g [3,6, "ASSAY"] = [3,4,5,6]
  # Creates a list between the ranges od the starting and ending match character
  '''

  dlis=[] 
  for i in range (0, len(lis)):  
    for j in range (0, len(lis[0])):
      dlis.append(list(range(lis[i][j][0],lis[i][j][1],1)))  
              
  #********************************************************

  dlis = set ( tuple (x) for x in dlis) # clearning out repetitions
  dlis = list (list (x) for x in dlis)  # back to list to access elements by indexing
  #no repeating arrays
  dlis=(sorted(dlis, key=len)) # sort inner arrays by length
  
  to_remove=[] # list of elements needed to be removed
  ind=[]   # list of indices of dlis that need to be removed

  ###############################################################################
    # On to detecting overlapping matches
  for i in range(0, len(dlis)):    # calucalte Jacardi index between evey element and each forward element
    for j in range(i+1,len(dlis)):   # if there is an overlap, keep the longest 
        #print(dlis[i],dlis[j])
        J=len(set(dlis[i]) & set(dlis[j])) / len(set(dlis[i]) | set(dlis[j]))
        #print(J)
        if (J!=0.0):   # if there is an overlap, write the element that should be removed 
           to_remove.append(dlis[i])
           ind.append(i)    # it is the i-th element because the list is sorted
           break # If it is similar with at least one, break the cycle
        
  result_list=[]
  for i in range (0 , len(dlis)):
     if (i not in ind):    # if "i" is not in "to_remove" then append it to results
       result_list.append(dlis[i])  # taking the indecies
   
   
   # just create a nested list because that's the needet output format 
   #
  out=[]
  for i in range (len(result_list)): 
    out.append([]) 

   # adapt the output to make it in the right format
   # and turn the sets back to [start,end,label] format
  label='ASSAY'  
  for i in range (0, len(result_list)):
      out[i].append(result_list[i][0])
      out[i].append(result_list[i][0] + len(result_list[i]))
      out[i].append(label)
  out = sorted (out)
  
  return out

z=21
print(tmp[z])
print("The processed output is:")
print(jac(tmp[z]))
print(type(test[1][1]))

# MEsto test mozda train za da ne se editne
for i in range (0,len(test)):
  test[i][1].update( entities = jac(tmp[i]) )

#for i in range (0, len(test)):
  #print("New data:" , test[i][1] )

print(test)   
data_train=test

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[84, 87, 'ASSAY'], [108, 111, 'ASSAY']]
The processed output is:
[[84, 87, 'ASSAY'], [108, 111, 'ASSAY']]
<class 'dict'>
[['1H-NMR-Based Metabolomic Study for Identifying Serum Profiles Associated with t...', {'entities': [[0, 6, 'ASSAY']]}], ['1H-NMR-Based Metabolomic Study for Identifying Serum Profiles Associated with the Response to Etanercept in Patients with Rheumatoid ArthritisPLoS ONE.', {'entities': [[0, 6, 'ASSAY']]}], ['Discriminating high-risk cervical Human Papilloma Virus infections with urinary biomarkers via non-targeted GC-MS-based metabolomicsPLoS ONE.', {'entities': [[111, 113, 'ASSAY']]}], ['2018; 13(12) Prognosis Biomarkers of Severe Sepsis and Septic Shock by 1H NMR Urine Metabolom...', {'entities': [[71, 77, 'ASSAY']]}], ['Prognosis Biomarkers of Severe Sepsis and Septic Shock by 1H NMR Urine Metabolomics in the Intensive Care', {'enti

In [63]:
#6
# Traning additional entity type using Spacy
# Because such a traning would need a lot of formating data, it was done using 
# a only one example, the NMR assay. Bigger and formated data set would be needed for a better model
#!/usr/bin/env python
# coding: utf8
"""Example of training an additional entity type

This script shows how to add a new entity type to an existing pretrained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.

The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.

After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.1.0+
Last tested with: v2.2.4
"""
from __future__ import unicode_literals, print_function

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy import displacy

# new entity label
LABEL = "ASSAY"

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting

# Train_data= [ ("tekst1",{"entities":[(start_char,end_char, LABEL)]},) , ("tekst2",{"entities":[(start_char,end_char, LABEL)]},) , ... ]
# In Word, do a search/replace, replacing all spaces with paragraph breaks (^p). Select all the text and then copy/paste it into Excel.

TRAIN_DATA = data_train  # data train is the one from the opened json file
              
#en_core_web_sm
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model="en_core_web_sm", new_model_name="ASSAY", output_dir=None, n_iter=35):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
    
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = ('NMR Metabolomics is used to determine the metabolic profile of biological samples, identify specific biomarkers,' 
       'and explore possible metabolic pathways. It has been used during drug development [1], '
       'and in clinical disease research [2, 3], pathology [4], toxicology [5] and nutrition studies [6].' 
       'Metabolomics mainly utilizes NMR spectroscopy [7], liquid chromatography (LC)–mass spectrometry [8] and ' 
       'gas chromatography–mass spectrometry [9] to analyze and evaluate biological specimens. '
       'Each analytical technique has its own advantages and shortcomings. none of them can be used individually' 
       'to systematically and accurately identify metabolites in complex biological matrices. '
       'Since accurate metabolite identification directly determines the usefulness of the metabolomic analysis,' 
       'metabolite identification has gained increased attention from the metabolomics research community. '
       '1H NMR spectroscopy is often used for metabolomics research. As all 1H nucleuses have the same sensitivity,' 
       'the reproducibility of NMR spectroscopy is typically high.')  
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    #ents = [ent for ent in list(doc.ents) if ent.label_ == 'ASSAY'] # trying to disable other entities but not working
    for ent in doc.ents:        
          print(ent.label_, ent.text)
    displacy.render(doc, style="ent",jupyter=True)
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

if __name__ == "__main__":
    x=main()
    

Loaded model 'en_core_web_sm'
Losses {'ner': 9407.517137901705}
Losses {'ner': 8713.402289576357}
Losses {'ner': 8242.427278756033}
Losses {'ner': 8192.706165627867}
Losses {'ner': 8020.404950931668}
Losses {'ner': 8008.3627757281065}
Losses {'ner': 8011.043619979304}
Losses {'ner': 7894.566299546277}
Losses {'ner': 7839.690645961091}
Losses {'ner': 7930.880325779319}
Losses {'ner': 7899.402653617552}
Losses {'ner': 7817.054569201544}
Losses {'ner': 7819.733863943897}
Losses {'ner': 7866.980376549065}
Losses {'ner': 7776.290685646236}
Losses {'ner': 7753.479310523719}
Losses {'ner': 7571.179981552908}
Losses {'ner': 7870.194521646481}
Losses {'ner': 7737.129519006936}
Losses {'ner': 7631.79074918438}
Losses {'ner': 7570.854939341545}
Losses {'ner': 7807.016919463174}
Losses {'ner': 7835.121870201256}
Losses {'ner': 7725.883611432335}
Losses {'ner': 7775.05771268066}
Losses {'ner': 7796.743461849401}
Losses {'ner': 7608.068307725116}
Losses {'ner': 7800.594547480345}
Losses {'ner': 7795

In [24]:

# -----------------------------------------------
# Make text into sentences the hard way but
# Computationally LESS expensive than Spacy 
# -----------------------------------------------
# 
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|ai|edu)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences



In [76]:
#2
import os
from bs4 import BeautifulSoup
#import urllib2
content=[]
train=[]
#url = "/content/drive/My Drive/Imperial college London/data/PMC3908470.html"
path = r"/content/drive/My Drive/Imperial college London/data"
for filename in os.listdir(path):
    if filename.endswith(".json"):
       fullpath = os.path.join(path, filename)
       with open(fullpath, "r") as read_file:   # read json file 
        data_train = json.load(read_file) 
        temp=data_train.get("paragraphs")
        for i in range(0,len(temp)):        #temp is list of dictiionaries under the key "Paragraphs"
          if (temp[i].get("section_heading")=="Materials and Methods"):  # PMC specific 
            train.append(temp[i].get("body")) #train is a list of all body sections

#print(data_train,type(data_train))
print((train))  
'''
# Turn to STRING and add to section 4
'''
qw=split_into_sentences('NMR Metabolomics is used to determine the metabolic profile of biological samples, identify specific biomarkers,' 
       'and explore possible metabolic pathways. It has been used during drug development [1], '
       'and in clinical disease research [2, 3], pathology [4], toxicology [5] and nutrition studies [6].' 
       'Metabolomics mainly utilizes NMR spectroscopy [7], liquid chromatography (LC)–mass spectrometry [8] and ' 
       'gas chromatography–mass spectrometry [9] to analyze and evaluate biological specimens. '
       'Each analytical technique has its own advantages and shortcomings. none of them can be used individually' 
       'to systematically and accurately identify metabolites in complex biological matrices. '
       'Since accurate metabolite identification directly determines the usefulness of the metabolomic analysis,' 
       'metabolite identification has gained increased attention from the metabolomics research community. '
       '1H NMR spectroscopy is often used for metabolomics research. As all 1H nucleuses have the same sensitivity,' 
       'the reproducibility of NMR spectroscopy is typically high.')
'''
#so huggin face proba
#!pip install spacy-transformers
#!python3 -m spacy download en_trf_bertbaseuncased_lg
import en_trf_bertbaseuncased_lg
import spacy
nlp = en_trf_bertbaseuncased_lg.load()
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")

print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)
'''
#print(type(qw))

['The DHS is a family-based study examining risk for macrovascular and other complications in T2D. Briefly, the DHS includes siblings concordant for T2D but without advanced renal insufficiency. When possible, unaffected siblings were also recruited. T2D was clinically defined as diabetes developing after the age of 35 years and initially treated with oral agents and/or diet and exercise, in the absence of historical evidence of ketoacidosis. Diagnoses were confirmed by measurement of fasting blood glucose and glycosylated hemoglobin (HbA1c). Extensive measurements of cardiovascular disease (CVD) risk factors were obtained during baseline exams, which occurred from 1998 to 2006. Ascertainment and recruitment have been previously described in detail [, ].', 'The DHS MIND is an ancillary study to the DHS that included a cognitive testing component and brain magnetic resonance imaging (MRI). The purpose was to investigate the relationships between cognitive function, brain imaging, and va

'\n#so huggin face proba\n#!pip install spacy-transformers\n#!python3 -m spacy download en_trf_bertbaseuncased_lg\nimport en_trf_bertbaseuncased_lg\nimport spacy\nnlp = en_trf_bertbaseuncased_lg.load()\ndoc = nlp("Apple shares rose on the news. Apple pie is delicious.")\n\nprint(doc[0].similarity(doc[7]))\nprint(doc._.trf_last_hidden_state.shape)\n'